In [19]:
!singularity exec \
    --cleanenv \
    --containall \
    -H $PWD:/home \
    --nv \
    stress_slurm/gan.sif \
    /bin/sh -c 'export TF_CPP_MIN_LOG_LEVEL=2 && python3.8 main.py --id 1'
    #/bin/sh -c 'export TF_CPP_MIN_LOG_LEVEL=2 && python3.8 main.py --id 5' #--saving'
    # /bin/sh -c 'export TF_CPP_MIN_LOG_LEVEL=2 && python3.8 noise_attack.py'

diff:  [-103.48647751  -10.36659427 -151.6013313     2.02915752  135.00308564
 -217.55387657   24.74438562 -282.97822814 -222.57102238   81.56061255
   -1.40755831  308.61879177 -104.78686192  -37.28388735 -180.38462623
    4.21952392  -38.21340629 -209.68826684   76.74115469 -201.23149072
  -92.94265462 -336.11375745 -214.3071488   146.60203974   73.48624666
 -114.65600383 -144.42887529   50.21241279  -88.12743738 -196.70160911
  -81.2691649   237.46501318 -196.99058105   65.24541602   99.75600243
   81.53805535 -143.25435563  -64.55601569  181.51526475  -89.82332181
   15.41619057  -73.02280647   91.17431312  -98.65916279  -29.86528689
 -129.61485145  -89.38548306  107.82337734 -210.31576072  -74.15114389
  -94.9749087   473.58019487 -147.61444668 -108.67681954  -94.87301396
  -37.72069593  121.08848401  -11.37718166  -11.15558552   -9.70119666
 -143.74489679 -223.21059288   35.19923137   92.59394082  142.59674638
 -103.68286368   -7.188937     -1.26431809   16.64540732  175.55325154

In [14]:
#!cd stress_slurm \
#    && singularity exec gan.sif spython recipe Dockerfile > Singularity \
#    && singularity build gan_.sif Singularity